In [1]:
#mount your google drive to avoid losing data on colab when session expires 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''Detecting number of occupied parking spots. Code written keeping in mind the resources provided
   under google colab 
'''
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
import torch

In [2]:
import glob
import os

os.chdir("/content/drive/MyDrive/vehicle_detection/Smart-Park/Smart-Park-with-YOLO-V3")
# Directory of images to run detection on
ROOT_DIR = os.getcwd()
print(ROOT_DIR)

/content/drive/MyDrive/vehicle_detection/Smart-Park/Smart-Park-with-YOLO-V3


In [3]:
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
import pandas as pd
import random 
import pickle as pkl
from darknet import Darknet
from preprocess import inp_to_image, letterbox_image
from util import *

In [ ]:
!pip install Cython
!git clone https://github.com/waleedka/coco
!pip install -U setuptools
!pip install -U wheel
os.getcwd()

In [4]:
def get_test_input(input_dim, CUDA):
    img = cv2.imread("/content/drive/MyDrive/vehicle_detection/Smart-Park/Smart-Park-with-YOLO-V3/testFrame_02.jpg")
    img = cv2.resize(img, (input_dim, input_dim)) 
    img_ =  img[:,:,::-1].transpose((2,0,1))
    img_ = img_[np.newaxis,:,:,:]/255.0
    img_ = torch.from_numpy(img_).float()
    img_ = Variable(img_)
    
    if CUDA:
        img_ = img_.cuda()
    
    return img_

In [5]:
def prep_image(img, inp_dim):
    """
    Prepare image for inputting to the neural network. 
    Returns a Variable 
    """
    orig_im = img
    dim = orig_im.shape[1], orig_im.shape[0]
    img = (letterbox_image(orig_im, (inp_dim, inp_dim)))
    img_ = img[:,:,::-1].transpose((2,0,1)).copy()
    img_ = torch.from_numpy(img_).float().div(255.0).unsqueeze(0)
    return img_, orig_im, dim

In [6]:
def write(x, img, classes):
    colors = [(0,0,255), (255,255,255), (255,0,0), (0,255,0), (0,255,255), 
              (255,255,0), (205,120,110), (23,18,150), (135,120,1), (1,182,50)]
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    cls = int(x[-1])
    label = "{0}".format(classes[cls])
    color = (0, 0, 0)
    if cls <= 9:
      color = colors[cls]
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img

In [7]:
    print("Loading network.....")
    model = Darknet("/content/drive/MyDrive/vehicle_detection/Smart-Park/Smart-Park-with-YOLO-V3/cfg/yolov3.cfg")
    model.load_weights("/content/drive/MyDrive/vehicle_detection/Smart-Park/yolov3.weights")
    print("Network successfully loaded")

Loading network.....
Network successfully loaded


In [98]:
def frame_postprocess(frames,inp_dim,CUDA,confidence,num_classes,nms_thesh,frame_count,batch_size):
    for i, item in enumerate(frames):
         frame = item
         img, orig_im, dim = prep_image(frame, inp_dim)
         im_dim = torch.FloatTensor(dim).repeat(1,2)                        
         if CUDA:
            im_dim = im_dim.cuda()
            img = img.cuda()
         with torch.no_grad():   
              output = model(Variable(img), CUDA)
         output = write_results(output, confidence, num_classes, nms = True, nms_conf = nms_thesh)
         print("count cars {}".format(output.size(0)))
         im_dim = im_dim.repeat(output.size(0), 1)
         scaling_factor = torch.min(inp_dim/im_dim,1)[0].view(-1,1)
         output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
         output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2
         output[:,1:5] /= scaling_factor
         for i in range(output.shape[0]):
                output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
                output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])
         classes = load_classes('/content/drive/MyDrive/vehicle_detection/Smart-Park/Smart-Park-with-YOLO-V3/data/coco.names')
         colors = pkl.load(open("/content/drive/MyDrive/vehicle_detection/Smart-Park/Smart-Park-with-YOLO-V3/pallete", "rb"))
         list(map(lambda x: write(x, orig_im, classes), output))
         n_vehicles = output.size(0)
         cv2.putText(orig_im, "Total vehicles detected: " + str(n_vehicles), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, [0,0,0], 3, cv2.LINE_AA)
         name = '{0}.jpg'.format(frame_count + i - batch_size)
         name = os.path.join('./ak', name)
         cv2.imwrite(name, frame)
         key = cv2.waitKey(1)
         if key & 0xFF == ord('q'):
              break

In [105]:
def yolo_videoprocess(video):
    batch_size = 30
    confidence = 0.5
    nms_thesh = float(0.4)
    start = 0

    num_classes = 80
    CUDA = torch.cuda.is_available()
    model.net_info["height"] = 704
    inp_dim = int(model.net_info["height"])
    #uncomment this if error occurs due to frame size
#     assert inp_dim % 32 == 0 
#     assert inp_dim > 32

    if CUDA:
        model.cuda()
        
    model(get_test_input(inp_dim, CUDA), CUDA)
    model.eval()
    videofile = video
    cap = cv2.VideoCapture(videofile)
    assert cap.isOpened(), 'Cannot capture source'
    frame_count = 0
    frames = []
    start = time.time()    
    while True:
        ret, frame = cap.read()
        if ret:
            frame_count += 1
            frames.append(frame)
            if len(frames) == batch_size:
              frame_postprocess(frames,inp_dim,CUDA,confidence,num_classes,nms_thesh,frame_count,batch_size)
              print("FPS of the video is {:5.2f}".format( frame_count / (time.time() - start)))
              frames = []
        else:
            break
    images = list(glob.iglob(os.path.join('./ak', '*.*')))
    images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))
    make_video('./out.mp4', images, fps=5)

In [97]:
videofile = "/content/drive/MyDrive/vehicle_detection/roadTrafficTest.mp4"
video = cv2.VideoCapture(videofile)

# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
if int(major_ver)  < 3 :
    fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else :
    fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

video.release();

Frames per second using video.get(cv2.CAP_PROP_FPS) : 29.97002997002997


In [63]:
# Get all image file paths to a list.
# Sort the images by name index.
# images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

In [103]:
!rm -r ak

In [104]:
!mkdir ak #don't execute remove and make dir in the same cell on colab, it creates error

In [106]:
yolo_videoprocess("/content/drive/MyDrive/vehicle_detection/roadTrafficTest.mp4")  #main program for detecting number of cars in the frames

count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
FPS of the video is  4.24
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
FPS of the video is  4.31
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1
count cars 1